## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
## determine which models are not already fit ##
import models_to_be_fit
num_of_p_values = 1
num_of_prediction_months = len(prediction_months)
churn_risk_prediction_exports = 'data/'+date_dir+'/exports/churn_risk_prediction/'+model_type+'/'
coefficients_and_pvalues_exports = 'data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'
(already_fit, yet_to_be_fit) = models_to_be_fit.main(model_type = model_type,\
                                                     model_numbers=model_numbers,\
                                                     date_dir=date_dir,\
                                                     churn_risk_prediction_exports=churn_risk_prediction_exports,\
                                                     coefficients_and_pvalues_exports=coefficients_and_pvalues_exports,\
                                                     data_set_name=data_set_name,\
                                                     num_of_p_values=num_of_p_values,\
                                                     num_of_prediction_months=num_of_prediction_months)

# for model_number in properly_used_inquiries_models['model_number'].unique():
for model_number in yet_to_be_fit:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)
    
    churn_risk_calculation.calculate_churn_risk(date_of_analysis = date_of_analysis, df_timeline = df_timeline, \
                                            vars_ = list(ctv.summary.index), data_all_spots = data_all_spots, ctv = ctv, \
                                            model_type = model_type, model_number = model_number,\
                                            with_add_vars = False, p_limit = p_limit,\
                                            event_date_type=event_date_full_name, spots_set = spots_set, with_wo_CB=with_wo_CB)
    
    for prediction_month in prediction_months:
        testing_results = model_testing.\
        get_testing_results(date_of_analysis=date_of_analysis,\
                            df_timeline=df_timeline.copy(), \
                            base_df = base_df.copy(),\
                            data_all_spots=data_all_spots.copy(), \
                            hs_list_path=hs_list_path,\
                            model_type=model_type,\
                            model_number=model_number,\
                            with_add_vars=False, p_limit=p_limit, event_date_type=event_date_full_name, \
                            prediction_month=prediction_month, ctv=ctv, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, penalizer=penalizer, test_with_req_canc_before=True)

        model_testing.save_results(date_of_analysis=date_of_analysis,\
                               df=testing_results[0], \
                               results_name='testing_results', \
                               model_type=model_type,\
                               model_number=model_number,\
                               spots_set=spots_set,\
                               with_wo_CB=with_wo_CB,\
                               event_date_type=event_date_full_name,\
                               with_add_vars=False, p_limit=p_limit, prediction_month=prediction_month)
        display(Markdown("## Model "+ str(model_number) + ": testing results"))
        display(testing_results[0].set_index('index'))

# Model 38: properly_used_private_parties_inquiries

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 38895 periods, 3017 subjects, 331 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 3017
 number of periods = 38895
  number of events = 331
partial log-likelihood = -2105.51
  time fit was run = 2023-02-04 08:06:41 UTC

---
                                                                                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                                             
Annual Commitment with 2 semi-annual payments                                  -0.29       0.75       0.38            -1.02             0.45                 0.36                 1.57
Annual Prepay                                                                   0.44       1.56       0.14             0.17             0.72                 1.18                 2.06
Fb.page.likes.analysis.avg.log2                                                -0.01       0.99       0.02            -0.04             0.03                 0.96                 1.03
Monthly                                                                         0.41       1.51       0.26            -0.09             0.91                 0.91                 2.49
New.email.subscriber.signups.last.month.div10                                  -0.01       0.99       0.04            -0.10             0.07                 0.91                 1.07
New.email.subscribers.allinclusive.last.month.div10                             0.01       1.01       0.02            -0.04             0.05                 0.96                 1.05
Opted.out.of.facebook                                                           0.11       1.12       0.25            -0.38             0.61                 0.69                 1.84
SpotHopper_site_not_live_yet                                                    0.54       1.71       0.31            -0.07             1.15                 0.93                 3.16
Website.Views.last.month.total.log2                                            -0.23       0.80       0.05            -0.32            -0.13                 0.72                 0.88
had_added_events_manually_edited_events_before_and_didnt_last_4_months          0.12       1.13       0.18            -0.23             0.47                 0.79                 1.60
had_added_events_manually_edited_events_last_4_months                          -0.33       0.72       0.22            -0.77             0.10                 0.47                 1.11
had_added_food_edited_food_before_and_didnt_last_4_months                       0.00       1.00       0.16            -0.31             0.32                 0.73                 1.37
had_added_food_edited_food_last_4_months                                       -0.12       0.89       0.18            -0.47             0.24                 0.62                 1.26
had_added_specials_edited_specials_before_and_didnt_last_4_months              -0.11       0.90       0.14            -0.39             0.17                 0.68                 1.19
had_added_specials_edited_specials_last_4_months                               -0.10       0.90       0.21            -0.52             0.32                 0.60                 1.37
had_bday_club_downloads_before_and_didnt_last_2_months                         -0.27       0.76       0.79            -1.82             1.28                 0.16                 3.59
had_bday_club_downloads_last_2_months                                           0.22       1.25       0.49            -0.74             1.18                 0.48                 3.26
had_catering_submissions_before_and_didnt_last_4_months                         0.31       1.36       0.30            -0.28             0.89                 0.76                 2.43
had_catering_submissions_last_4_mon

## without categorical variables with |coeff| < 0.2 and p value > 0.2

## without numerical variables with |coeff| < 0.01 and p value > 0.2

...

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,3763,3701
requested_cancellation,36,17
prediction_month,2022-11-01,2022-11-01
pearsons_corr_coef_and_p_value,"(0.028, 0.088)","(0.013, 0.413)"
spearmans_corr_coef_and_p_value,"(0.045, 0.006)","(0.038, 0.02)"
log_loss,0.063539,0.031559
concordance,63.389,66.285
discordance,36.611,33.715
requested_cancellation_in_the_first_20_%_of_all_spots,13,7


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,4053,4010
requested_cancellation,39,18
prediction_month,2022-12-01,2022-12-01
pearsons_corr_coef_and_p_value,"(0.043, 0.006)","(0.068, 0.0)"
spearmans_corr_coef_and_p_value,"(0.043, 0.006)","(0.061, 0.0)"
log_loss,0.063067,0.027254
concordance,62.718,76.371
discordance,37.282,23.629
requested_cancellation_in_the_first_20_%_of_all_spots,13,9


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,4327,4305
requested_cancellation,30,8
prediction_month,2023-01-01,2023-01-01
pearsons_corr_coef_and_p_value,"(0.107, 0.0)","(0.109, 0.0)"
spearmans_corr_coef_and_p_value,"(0.08, 0.0)","(0.069, 0.0)"
log_loss,0.039777,0.010304
concordance,77.67,96.434
discordance,22.33,3.566
requested_cancellation_in_the_first_20_%_of_all_spots,20,8
